In [9]:
import os
import math
import random
import time
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Input, Conv2D, MaxPooling2D, Dropout, Rescaling
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.vgg16 import VGG16
from hyperopt import hp, fmin, tpe, rand, STATUS_OK, Trials
from sklearn.metrics import accuracy_score

print(f"TensorFlow Version: {tf.__version__}")

# --- 1. DATA LOADING (SPARSE / INTEGER MODE) ---
# This mode is safer and fixes the Shape Mismatch error
TARGET_SIZE = (224, 224)
INPUT_SHAPE = (224, 224, 3)
BATCHSIZE = 32

# Load Training Data (Int Mode)
train_raw = image_dataset_from_directory(
    './train_224/',
    image_size=TARGET_SIZE,
    batch_size=BATCHSIZE,
    label_mode='int',  # <--- Changed to 'int' to match your error shape (None, 1)
    shuffle=True,
    seed=123
)

# Load Validation Data (Int Mode)
val_raw = image_dataset_from_directory(
    './test_224/',
    image_size=TARGET_SIZE,
    batch_size=BATCHSIZE,
    label_mode='int',  # <--- Changed to 'int'
    shuffle=False
)

class_names = train_raw.class_names
NUM_CLASSES = len(class_names)
print(f"Classes: {class_names} (Total: {NUM_CLASSES})")

# Normalization
normalization_layer = Rescaling(1./255)
train_ds = train_raw.map(lambda x, y: (normalization_layer(x), y)).cache().prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_raw.map(lambda x, y: (normalization_layer(x), y)).cache().prefetch(buffer_size=tf.data.AUTOTUNE)

# --- 2. CALLBACKS ---
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = {'batch':[], 'epoch':[]}
        self.accuracy = {'batch':[], 'epoch':[]}
        self.val_loss = {'batch':[], 'epoch':[]}
        self.val_acc = {'batch':[], 'epoch':[]}
    def on_batch_end(self, batch, logs={}):
        self.losses['batch'].append(logs.get('loss'))
        self.accuracy['batch'].append(logs.get('accuracy'))
        self.val_loss['batch'].append(logs.get('val_loss'))
        self.val_acc['batch'].append(logs.get('val_accuracy'))
    def on_epoch_end(self, batch, logs={}):
        self.losses['epoch'].append(logs.get('loss'))
        self.accuracy['epoch'].append(logs.get('accuracy'))
        self.val_loss['epoch'].append(logs.get('val_loss'))
        self.val_acc['epoch'].append(logs.get('val_accuracy'))
    def loss_plot(self, loss_type):
        iters = range(len(self.losses[loss_type]))
        plt.figure()
        if loss_type == 'epoch':
            plt.plot(iters, self.accuracy[loss_type], 'r', label='train acc')
            plt.plot(iters, self.losses[loss_type], 'g', label='train loss')
            plt.plot(iters, self.val_acc[loss_type], 'b', label='val acc')
            plt.plot(iters, self.val_loss[loss_type], 'k', label='val loss')
        plt.show()

history_this = LossHistory()

# --- 3. MODELS (COMPATIBLE WITH INTEGERS) ---

def vgg16(num_class, epochs=20, frozen=15, lr=0.001, patience=2, dropout_rate=0.5, verbose=1, savepath='./VGG16.h5', history=history_this, input_shape=INPUT_SHAPE):
    # Check for weights file
    weights_path = './vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'
    if os.path.exists(weights_path):
        base_model = VGG16(include_top=False, weights=weights_path, input_shape=input_shape)
    else:
        # Fallback to download if local file missing
        print("Local weights not found, downloading...")
        base_model = VGG16(include_top=False, weights='imagenet', input_shape=input_shape)

    for layer in base_model.layers[:frozen]:
        layer.trainable = False
    for layer in base_model.layers[frozen:]:
        layer.trainable = True

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(dropout_rate)(x)
    predictions = Dense(num_class, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions, name='vgg16')
    opt = Adam(learning_rate=lr)

    # CRITICAL FIX: Using 'sparse_categorical_crossentropy' handles (None, 1) targets automatically
    model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

    earlyStopping = callbacks.EarlyStopping(monitor='val_accuracy', patience=patience, verbose=verbose, mode='max')
    saveBestModel = callbacks.ModelCheckpoint(filepath=savepath, monitor='val_accuracy', verbose=verbose, save_best_only=True, mode='max')

    hist = model.fit(
        train_ds,
        epochs=epochs,
        validation_data=val_ds,
        callbacks=[earlyStopping, saveBestModel, history],
        verbose=verbose
    )
    return hist

# --- 4. PREDICTION ---
def prediction(model_path):
    model = load_model(model_path)
    rootdir = './test_224/'
    test_labels = []
    test_images = []

    # Walk directories
    for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            if file.lower().endswith((".jpeg", ".jpg", ".png")):
                label_name = subdir.split('/')[-1]
                test_labels.append(label_name)
                test_images.append(os.path.join(subdir, file))

    predict_results = []
    class_indices = {name: i for i, name in enumerate(class_names)}
    index_to_label = {v: k for k, v in class_indices.items()}

    print(f"Predicting {len(test_images)} images...")

    for img_path in test_images:
        try:
            img = Image.open(img_path)
            img = img.resize(TARGET_SIZE)
            img_array = np.array(img) / 255.0  # Normalize
            img_array = np.expand_dims(img_array, axis=0)

            # Predict
            pred_probs = model.predict(img_array, verbose=0)
            pred_class_id = np.argmax(pred_probs, axis=1)[0] # Returns integer index

            predict_results.append(index_to_label[pred_class_id])
        except Exception as e:
            print(f"Error on {img_path}: {e}")
            predict_results.append("error")

    return accuracy_score(test_labels, predict_results)

# --- 5. HYPEROPT ---
def objective(params):
    print(f"Testing: {params}")
    try:
        vgg16(num_class=NUM_CLASSES,
              frozen=int(params['frozen']),
              epochs=int(params['epochs']),
              patience=int(params['patience']),
              lr=params['lr'],
              dropout_rate=params['dropout_rate'],
              verbose=0)

        acc = prediction('./VGG16.h5')
        print(f'Accuracy: {acc}')
        return {'loss': -acc, 'status': STATUS_OK}
    except Exception as e:
        print(f"Trial failed: {e}")
        return {'loss': 0, 'status': STATUS_OK} # Fail gracefully

space = {
    'frozen': hp.quniform('frozen', 15, 18, 1),
    'epochs': hp.quniform('epochs', 5, 8, 1),
    'patience': hp.quniform('patience', 2, 4, 1),
    'lr': hp.loguniform('lr', np.log(0.0001), np.log(0.01)),
    'dropout_rate': hp.uniform('dropout_rate', 0.3, 0.6),
}

# Run
t1 = time.time()
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=2)
print(f"Best: {best}")

TensorFlow Version: 2.20.0
Found 287 files belonging to 5 classes.
Found 75 files belonging to 4 classes.
Classes: ['0', '1', '2', '3', '4'] (Total: 5)
Testing: {'dropout_rate': 0.4829415228923738, 'epochs': 7.0, 'frozen': 18.0, 'lr': 0.0016010353790399281, 'patience': 3.0}
  0%|          | 0/2 [00:00<?, ?trial/s, best loss=?]

Predicting 75 images...                              
Accuracy: 1.0                                        
Testing: {'dropout_rate': 0.4978189394165963, 'epochs': 5.0, 'frozen': 16.0, 'lr': 0.00011515140064929637, 'patience': 2.0}
 50%|█████     | 1/2 [05:26<05:26, 326.37s/trial, best loss: -1.0]

Predicting 75 images...                                           
Accuracy: 1.0                                                     
100%|██████████| 2/2 [09:02<00:00, 271.08s/trial, best loss: -1.0]
Best: {'dropout_rate': np.float64(0.4829415228923738), 'epochs': np.float64(7.0), 'frozen': np.float64(18.0), 'lr': np.float64(0.0016010353790399281), 'patience': np.float64(3.0)}
